# Text-to-SQL: Evaluation and Optimization

This notebook evaluates Text-to-SQL performance and explores optimization strategies.

## Objectives
1. Evaluate model performance on test queries
2. Analyze error patterns and failure modes
3. Measure performance metrics (accuracy, latency, cost)
4. Optimize for production deployment

In [ ]:
import sys
import os
import json
import pandas as pd
import numpy as np
from pathlib import Path
from typing import List, Dict
import time
from collections import defaultdict
import sqlite3

# Add src to path
sys.path.append(str(Path('../src').resolve()))

from schema_manager import SchemaManager
from query_generator import TextToSQLGenerator, QueryResult
from query_validator import QueryValidator

# Set up matplotlib for visualizations
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

# Note: Set OPENAI_API_KEY environment variable for live testing

## 1. Load Test Data

In [ ]:
# Initialize components
db_path = "../data/sample_database.db"
validator = QueryValidator(db_path)

# Load test queries
with open('../data/test_queries.json', 'r') as f:
    test_queries = json.load(f)

print(f"Total test queries: {len(test_queries)}")
print(f"\nDistribution by complexity:")
complexity_dist = pd.Series([q['complexity'] for q in test_queries]).value_counts()
print(complexity_dist)

print(f"\nDistribution by category:")
category_dist = pd.Series([q['category'] for q in test_queries]).value_counts()
print(category_dist)

## 2. Validation Metrics

Define metrics for evaluating SQL query quality.

In [ ]:
def compare_queries(expected_sql: str, generated_sql: str, db_path: str) -> Dict:
    """
    Compare expected and generated SQL queries.
    
    Returns dict with:
    - syntax_valid: bool
    - results_match: bool
    - row_count_match: bool
    - column_names_match: bool
    """
    conn = sqlite3.connect(db_path)
    
    result = {
        'syntax_valid': False,
        'results_match': False,
        'row_count_match': False,
        'column_names_match': False,
        'error': None
    }
    
    try:
        # Check syntax
        validator = QueryValidator(db_path)
        validation = validator.validate_query(generated_sql)
        result['syntax_valid'] = validation['is_valid']
        
        if not result['syntax_valid']:
            result['error'] = validation.get('error')
            return result
        
        # Execute both queries
        expected_df = pd.read_sql_query(expected_sql, conn)
        generated_df = pd.read_sql_query(generated_sql, conn)
        
        # Compare row counts
        result['row_count_match'] = len(expected_df) == len(generated_df)
        
        # Compare column names
        result['column_names_match'] = list(expected_df.columns) == list(generated_df.columns)
        
        # Compare actual results
        if result['row_count_match'] and result['column_names_match']:
            # Sort both dataframes for comparison
            expected_sorted = expected_df.sort_values(by=list(expected_df.columns)).reset_index(drop=True)
            generated_sorted = generated_df.sort_values(by=list(generated_df.columns)).reset_index(drop=True)
            result['results_match'] = expected_sorted.equals(generated_sorted)
        
    except Exception as e:
        result['error'] = str(e)
    finally:
        conn.close()
    
    return result

# Test the comparison function
test_query = test_queries[0]
comparison = compare_queries(
    test_query['expected_sql'],
    test_query['expected_sql'],  # Should match itself
    db_path
)
print("Self-comparison test:")
print(comparison)

## 3. Offline Validation

Validate all expected SQL queries in the test set.

In [ ]:
# Validate all expected queries
validation_results = []

for query in test_queries:
    validation = validator.validate_query(query['expected_sql'])
    
    validation_results.append({
        'question': query['question'],
        'complexity': query['complexity'],
        'category': query['category'],
        'is_valid': validation['is_valid'],
        'is_safe': validation.get('is_safe', False),
        'tables_exist': validation.get('tables_exist', False),
    })

validation_df = pd.DataFrame(validation_results)
print("Expected Query Validation:")
print("=" * 80)
print(f"All valid: {validation_df['is_valid'].all()}")
print(f"All safe: {validation_df['is_safe'].all()}")
print(f"All tables exist: {validation_df['tables_exist'].all()}")

if not validation_df['is_valid'].all():
    print("\nInvalid queries:")
    print(validation_df[~validation_df['is_valid']])

## 4. Model Performance Evaluation (Requires API Key)

In [ ]:
# Example evaluation framework (uncomment and add API key to run)
"""
# Initialize generator
generator = TextToSQLGenerator(db_path, provider='openai', model_name='gpt-3.5-turbo')

# Evaluate on test set
evaluation_results = []

for query in test_queries:
    print(f"Evaluating: {query['question'][:50]}...")
    
    # Generate SQL
    start_time = time.time()
    result = generator.generate_sql(query['question'])
    latency = (time.time() - start_time) * 1000
    
    # Compare with expected
    comparison = compare_queries(query['expected_sql'], result.sql, db_path)
    
    evaluation_results.append({
        'question': query['question'],
        'complexity': query['complexity'],
        'category': query['category'],
        'expected_sql': query['expected_sql'],
        'generated_sql': result.sql,
        'status': result.status,
        'syntax_valid': comparison['syntax_valid'],
        'results_match': comparison['results_match'],
        'prompt_tokens': result.prompt_tokens,
        'completion_tokens': result.completion_tokens,
        'total_tokens': result.prompt_tokens + result.completion_tokens,
        'latency_ms': latency,
        'model_used': result.model_used,
        'error': comparison.get('error'),
    })
    
    time.sleep(1)  # Rate limiting

# Save results
eval_df = pd.DataFrame(evaluation_results)
eval_df.to_csv('evaluation_results.csv', index=False)

print("\nEvaluation complete! Results saved to evaluation_results.csv")
"""

print("Live evaluation disabled.")
print("To run evaluation: uncomment code above and set OPENAI_API_KEY")

# Load sample results for demonstration
# For now, create synthetic results
print("\nCreating synthetic results for demonstration...")

np.random.seed(42)
eval_df = pd.DataFrame([
    {
        'question': q['question'],
        'complexity': q['complexity'],
        'category': q['category'],
        'syntax_valid': np.random.choice([True, False], p=[0.95, 0.05]),
        'results_match': np.random.choice([True, False], p=[0.85, 0.15]),
        'total_tokens': np.random.randint(200, 800),
        'latency_ms': np.random.randint(500, 3000),
    }
    for q in test_queries
])

## 5. Performance Analysis

In [ ]:
# Overall accuracy metrics
print("Overall Performance Metrics:")
print("=" * 80)
print(f"Syntax Accuracy: {eval_df['syntax_valid'].mean():.1%}")
print(f"Result Accuracy: {eval_df['results_match'].mean():.1%}")
print(f"Avg Latency: {eval_df['latency_ms'].mean():.0f}ms")
print(f"Avg Tokens: {eval_df['total_tokens'].mean():.0f}")

# Performance by complexity
print("\nPerformance by Complexity:")
print("=" * 80)
complexity_metrics = eval_df.groupby('complexity').agg({
    'syntax_valid': 'mean',
    'results_match': 'mean',
    'latency_ms': 'mean',
    'total_tokens': 'mean',
}).round(3)
print(complexity_metrics)

# Performance by category
print("\nPerformance by Category:")
print("=" * 80)
category_metrics = eval_df.groupby('category').agg({
    'syntax_valid': 'mean',
    'results_match': 'mean',
}).round(3)
print(category_metrics)

In [ ]:
# Visualize performance by complexity
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy by complexity
accuracy_data = eval_df.groupby('complexity')[['syntax_valid', 'results_match']].mean()
accuracy_data.plot(kind='bar', ax=axes[0], rot=0)
axes[0].set_title('Accuracy by Query Complexity')
axes[0].set_ylabel('Accuracy')
axes[0].set_ylim([0, 1])
axes[0].legend(['Syntax Valid', 'Results Match'])
axes[0].grid(axis='y', alpha=0.3)

# Latency and tokens by complexity
ax2 = axes[1]
complexity_order = ['simple', 'medium', 'complex']
latency_data = eval_df.groupby('complexity')['latency_ms'].mean().reindex(complexity_order)
ax2.bar(range(len(latency_data)), latency_data.values, alpha=0.7, label='Latency (ms)')
ax2.set_xticks(range(len(complexity_order)))
ax2.set_xticklabels(complexity_order)
ax2.set_ylabel('Latency (ms)', color='blue')
ax2.tick_params(axis='y', labelcolor='blue')
ax2.set_title('Latency and Token Usage by Complexity')

ax2_twin = ax2.twinx()
token_data = eval_df.groupby('complexity')['total_tokens'].mean().reindex(complexity_order)
ax2_twin.plot(range(len(token_data)), token_data.values, 'ro-', linewidth=2, markersize=8, label='Tokens')
ax2_twin.set_ylabel('Total Tokens', color='red')
ax2_twin.tick_params(axis='y', labelcolor='red')

plt.tight_layout()
plt.savefig('performance_by_complexity.png', dpi=150, bbox_inches='tight')
plt.show()

print("Chart saved as 'performance_by_complexity.png'")

## 6. Error Analysis

In [ ]:
# Identify failed queries
failed_queries = eval_df[~eval_df['results_match']]

print(f"Failed Queries: {len(failed_queries)} / {len(eval_df)} ({len(failed_queries)/len(eval_df):.1%})")
print("\nFailure Distribution:")
print("=" * 80)
print(f"By complexity: {failed_queries['complexity'].value_counts().to_dict()}")
print(f"By category: {failed_queries['category'].value_counts().to_dict()}")

# Show sample failed queries
if len(failed_queries) > 0:
    print("\nSample Failed Queries:")
    print("=" * 80)
    for idx, row in failed_queries.head(3).iterrows():
        print(f"\nQuestion: {row['question']}")
        print(f"Complexity: {row['complexity']}")
        print(f"Category: {row['category']}")
        print("-" * 80)

## 7. Cost Analysis

In [ ]:
# Estimate API costs (GPT-3.5-turbo pricing as of 2024)
COST_PER_1K_PROMPT_TOKENS = 0.0005  # $0.50 per 1M tokens
COST_PER_1K_COMPLETION_TOKENS = 0.0015  # $1.50 per 1M tokens

# Assuming 70/30 split for prompt/completion
eval_df['prompt_tokens_est'] = (eval_df['total_tokens'] * 0.7).astype(int)
eval_df['completion_tokens_est'] = (eval_df['total_tokens'] * 0.3).astype(int)

eval_df['cost_usd'] = (
    eval_df['prompt_tokens_est'] / 1000 * COST_PER_1K_PROMPT_TOKENS +
    eval_df['completion_tokens_est'] / 1000 * COST_PER_1K_COMPLETION_TOKENS
)

print("Cost Analysis:")
print("=" * 80)
print(f"Total tokens: {eval_df['total_tokens'].sum():,}")
print(f"Total cost: ${eval_df['cost_usd'].sum():.4f}")
print(f"Avg cost per query: ${eval_df['cost_usd'].mean():.6f}")
print(f"\nCost by complexity:")
print(eval_df.groupby('complexity')['cost_usd'].agg(['count', 'mean', 'sum']).round(6))

# Project monthly costs
queries_per_day = 1000
monthly_cost = eval_df['cost_usd'].mean() * queries_per_day * 30
print(f"\nProjected monthly cost (1000 queries/day): ${monthly_cost:.2f}")

## 8. Optimization Strategies

In [ ]:
# Caching analysis
# Identify duplicate or similar queries that could benefit from caching

print("Caching Opportunity Analysis:")
print("=" * 80)

# Simulate cache hit rate for exact matches
cache_hit_rate = 0.15  # Assume 15% of queries are duplicates
queries_without_cache = len(eval_df)
queries_with_cache = int(queries_without_cache * (1 - cache_hit_rate))
cost_savings = eval_df['cost_usd'].sum() * cache_hit_rate

print(f"Without cache: {queries_without_cache} API calls")
print(f"With cache (15% hit rate): {queries_with_cache} API calls")
print(f"Cost savings: ${cost_savings:.4f} ({cache_hit_rate:.0%})")

# Model selection optimization
print("\nModel Selection Strategy:")
print("=" * 80)
print("Recommendation:")
print("- Simple queries: Use GPT-3.5-turbo (lower cost)")
print("- Medium queries: Use GPT-3.5-turbo with retry to GPT-4 if needed")
print("- Complex queries: Start with GPT-4 (higher accuracy)")
print("\nEstimated cost reduction: 30-40% with complexity-based routing")

In [ ]:
# Latency optimization
print("Latency Optimization:")
print("=" * 80)

p50_latency = eval_df['latency_ms'].quantile(0.5)
p95_latency = eval_df['latency_ms'].quantile(0.95)
p99_latency = eval_df['latency_ms'].quantile(0.99)

print(f"P50 latency: {p50_latency:.0f}ms")
print(f"P95 latency: {p95_latency:.0f}ms")
print(f"P99 latency: {p99_latency:.0f}ms")

print("\nOptimization strategies:")
print("1. Cache frequent queries (reduces latency to <10ms)")
print("2. Parallel processing for batch queries")
print("3. Use streaming for long responses")
print("4. Implement timeout and fallback mechanisms")
print(f"5. Set SLA target: P95 < {p95_latency * 0.8:.0f}ms")

## 9. Production Readiness Checklist

In [ ]:
# Production metrics summary
accuracy_threshold = 0.90
latency_threshold_p95 = 2000  # ms
cost_per_query_threshold = 0.01  # USD

meets_accuracy = eval_df['results_match'].mean() >= accuracy_threshold
meets_latency = p95_latency <= latency_threshold_p95
meets_cost = eval_df['cost_usd'].mean() <= cost_per_query_threshold

print("Production Readiness Assessment:")
print("=" * 80)
print(f"✓ Accuracy ≥ {accuracy_threshold:.0%}: {'PASS' if meets_accuracy else 'FAIL'} ({eval_df['results_match'].mean():.1%})")
print(f"✓ P95 Latency ≤ {latency_threshold_p95}ms: {'PASS' if meets_latency else 'FAIL'} ({p95_latency:.0f}ms)")
print(f"✓ Cost per query ≤ ${cost_per_query_threshold}: {'PASS' if meets_cost else 'FAIL'} (${eval_df['cost_usd'].mean():.6f})")

all_pass = meets_accuracy and meets_latency and meets_cost
print(f"\nOverall: {'✓ READY FOR PRODUCTION' if all_pass else '✗ NEEDS OPTIMIZATION'}")

if not all_pass:
    print("\nRecommended improvements:")
    if not meets_accuracy:
        print("- Improve prompt templates")
        print("- Add more few-shot examples")
        print("- Use GPT-4 for complex queries")
    if not meets_latency:
        print("- Implement query caching")
        print("- Optimize prompt length")
        print("- Use faster model for simple queries")
    if not meets_cost:
        print("- Implement tiered model selection")
        print("- Reduce token usage in prompts")
        print("- Increase cache hit rate")

## 10. Summary and Recommendations

### Performance Summary:
- **Accuracy**: Syntax validity and result matching rates by complexity
- **Latency**: P50/P95/P99 response times
- **Cost**: Per-query and projected monthly costs

### Key Optimizations:
1. **Query Caching**: 15-30% cost reduction for repeated queries
2. **Model Routing**: Use GPT-3.5 for simple queries, GPT-4 for complex ones
3. **Prompt Optimization**: Reduce token usage by 20-30% with compact schemas
4. **Batch Processing**: Handle multiple queries in parallel

### Production Deployment:
1. Set up monitoring for accuracy, latency, and cost
2. Implement circuit breakers and fallback mechanisms
3. Add comprehensive logging for debugging
4. Create alerting for quality degradation
5. Plan for A/B testing of prompt variations

### Next Steps:
- Deploy with gradual rollout (10% → 50% → 100% traffic)
- Monitor real-world performance vs. test results
- Collect user feedback on query quality
- Continuously update few-shot examples based on failures